### Get Input File & Setup

In [61]:
table=[
    [1,1,1,1,1],
    [1,1,1,0,1],
    [1,0,1,1,1],
    [0,1,1,1,1],
    [0,1,1,0,1],
    [1,0,1,0,0],
    [0,0,0,1,1],
    [0,0,0,0,0]
]

B = {
    'A','B','C','D','E'
}

FactorHash = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4
}

NegFactorSet = {}

### Perform Alg on File

In [63]:
#STEP 0

# FactorTable is a hashtable/dictionary of all factors, factorA -> index | key -> value
# For everything in W, we will have to test this
# Over every factor, we are collecting the set of factors that are possible effects, this set is W
# B: set of all factors, main_table: C
def step0(main_table, B, FactorTable, NegFactorSet):
    print(main_table)
    W = []
    for factor in B:
        print("Looking at " + factor)
        #(2)
        if rowDuplicity(main_table, FactorTable[factor]):
            print(factor + " failed row duplicity")
            continue
        #(3)
        elif isNeg(FactorTable[factor], NegFactorSet):
            print(factor + " failed isNeg")
            continue
        W.append(factor)
    return W

'''
    Check if the same row but negative factor already exists, if not, insert into set, else return 1
    int[i][j] table, i= coincidence, j = factor index
'''
def rowDuplicity(table, factor):
    rowSet = []
    #loop through every row in table
    for coincidence in table:
        modif_coincidence = coincidence
        modif_coincidence[factor] = !modif_coincidence[factor]
        if modif_coincidence in rowSet:
            return 1
        rowSet.append(coincidence)
        
def isNeg(factor, NegFactorSet):
    if factor in NegFactorSet:
        return 1
    return 0

In [64]:
print(step0(table, B, FactorHash, NegFactorSet))

Looking at C
C failed row duplicity
Looking at B
B failed row duplicity
Looking at D
D failed row duplicity
Looking at A
A failed row duplicity
Looking at E
E failed row duplicity
[]


### Main Loop

In [ ]:

#-------------------------------------------------------------------------------
#STEP 1
#FOR LOOP

#--------------------------------
#STEP 2

'''
table: int[][] : double array of 1/0
factor: int : index of Z_i 
coincidence: X_k Z_i
modif_coincidence: X_k !Z_i
returns set of rows where coincidence X_k is sufficient for Z_i
'''
def step2(main_table, factor):
    rowSet = {}
    rowNegSet = {}
    for coincidence in table:
        
        modif_coincidence = coincidence 
        modif_coincidence[factor] = !modif_coincidence[factor]
        
        if coincidence[factor]==1:   #if Z_i is true add to set of sufficient rows unless modif_coincidence is in the table
            if modif_coincidence not in rowNegSet:
                rowSet.add(coincidence)
                
        elif coincidence[factor]==0: #if Z_i is false remove modif_coincidence from sufficient rows
            if modif_coincidence in rowSet:
                rowSet.remove(coincidence)
            rowNegSet.add(coincidence)
    return rowSet
#--------------------------------
#STEP 3

from itertools import permutations

def step3(rowSet, main_table, effect)
    for coincidence in rowSet:
        prm_list = list(permutations(range(0,coincidence.length)))
        # for every permutation of coincidence
        for prm in prm_list:
            #remove Z_g
            for factor in prm:
                #remove
                removeprm()
                #is remaining prm suff?
                if !check_sufficient(coincidence, mod_prm, table, effect):
                    addbackprm()
                    
# returns 1 if sufficient
def check_sufficient(coincidence, prm, table, effect):
    extravar = false
    for row table:
        var = true
        for col in row:
            if col.index is in prm:
                if col == coincidence[col.index]:
                    
                else:
                    var = false
                    break
        if var is true:
            extravar = true
            #in order for the original coincidence to be sufficient, its effect had to have been 1
            if row[effect] == 0:
                return 0
    return extravar

#--------------------------------
#STEP 4
#END OF FOR LOOP -- WE ARE IGNORING THIS STEP FOR EFFICIENCY

#--------------------------------
#STEP 5

def step5(main_table, s_conditions, s_prms, effect):
    n_conditions = {}
    for s_cond in s_conditions:
        is_necessary = true
        for row in main_table:
            if row[effect] == 1:
                for index in s_prms:
                    if s_cond[index] != row[index]:
                        is_necessary = false
                        break
                if !is_necessary:
                    break
        if is_necessary:
            n_conditions.add(s_cond)
    return n_conditions             

#--------------------------------
#STEP 6

from itertools import permutations

def step6(rowSet, main_table, effect)
    for coincidence in rowSet:
        prm_list = list(permutations(range(0,coincidence.length)))
        # for every permutation of coincidence
        for prm in prm_list:
            #remove Z_g
            for factor in prm:
                #remove
                removeprm()
                #is remaining prm suff?
                if !check_necessary(table, coincidence, mod_prm, effect):
                    addbackprm()
                    
def check_necessary(main_table, coincidence, prm, effect):
    for row in main_table:
        if row[effect] == 1:
            for index in prm:
                if coincidence[index] != row[index]:
                    return false
    return true

### Framing